# LAB GenAI - LLMs - OpenAI Assistant

## Exercise 1

Create an assistant to answer a topic of your choosing:
 - Upload a file of your interest
 - Add Instructions to the prompt
 - Use the assistant in Playground mode

 https://platform.openai.com/playground/assistants

In [1]:
import openai
import pandas as pd

#OpenAI API key
api_key = ''

In [2]:
## Create the assistant
from openai import OpenAI
 
client = OpenAI(api_key=api_key)
 
assistant = client.beta.assistants.create(
  name="Application Assistant",
  instructions="You are an expert in writing job applications. If I send you a job description you need to write a cover letter suitable for that role based on my experiences taken from my CV.",
  model="gpt-4o",
  tools=[{"type": "file_search"}],
)

In [6]:
# Create a vector store caled "Application Documents"
vector_store = client.beta.vector_stores.create(name="Application Documents")

In [4]:
 # Ready the files for upload to OpenAI
file_paths = [""]
file_streams = [open(path, "rb") for path in file_paths]

In [7]:
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [8]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

## Exercise 2

Talk to your assistant via the API

https://platform.openai.com/docs/assistants/overview

In [9]:
# Create a thread and attach the file to the message
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Can you rewrite this 'Dear Recruiting Team,I am excited to apply for the Generative Machine Learning Engineer position at xxx. With a deep passion for augmented and virtual reality, and a newly discovered love for machine learning, I am drawn to the opportunity to merge these two areas to create transformative experiences. xxx's mission to redefine immersive journeys resonates with my ambition to contribute to innovative projects that merge cutting-edge AI and experiential design. Driven by a long-standing interest in immersive technologies, I was among the first to enroll in the AR/VR/XR Development and Design degree program when it was introduced. This decision was fueled by my fascination with the limitless potential of augmented and virtual reality to create new worlds and redefine how we perceive and interact with our surroundings. This passion, combined with my technical skills in 3D modeling, computer graphics, and development with Unity, has shaped my drive to explore and create in this dynamic field. Building on this foundation, I am currently completing a Data Science and Machine Learning Bootcamp, where I have developed a profound interest in leveraging machine learning to enhance and optimize immersive experiences. The bootcamp has equipped me with practical experience in building deep learning models with frameworks like PyTorch, deploying solutions on cloud platforms, and managing large datasets using SQL. My newly discovered enthusiasm for refining and optimizing models has driven my desire to merge my expertise in AR/VR/XR with the potential of machine learning. ZAUBAR offers an inspiring opportunity to contribute to a team that's pioneering immersive digital experiences. I am eager to apply my background in immersive technologies and machine learning to help develop innovative solutions that bring the metaverse to the streets. Thank you for considering my application. Yours sincerely,' to suit that exact position as an AI engineer 'SLAY, founded in December 2022, has swiftly made its mark in the mobile world. In 2023, we developed several apps and games that reached the #1 spot in the App Store across 3 continents. Backed by some of the world's most esteemed investors, including Accel, Scooter Braun, Scott Belsky, and Ilkka Paananen, SLAY is rapidly growing and reaching millions of users. We're looking to bring together the smartest individuals to achieve our mission to redefine the social gaming landscape. Tasks: Design and Implement AI Systems: Develop and implement the AI architecture from the ground up, focusing on creating a seamless and dynamic interaction between Character and the game environment. AI Modeling and Simulation: Build models for perception, intention, and action layers using advanced machine learning techniques. Integration of AI with Game Engine: Ensure smooth integration of AI models with the GoDot game engine, managing real-time data exchange and action execution within the game. Continuous Learning and Optimization: Implement mechanisms for real-time learning and memory updates using feedback from game interactions, improving Character behaviors and decision-making processes over time. Collaboration and Communication: Work closely with game developers, backend developers, and project managers to ensure that AI components align with overall game design and project goals. Performance Monitoring: Monitor and optimize the performance of AI systems, ensuring they meet the requirements for speed and efficiency in a real-time gaming context. Documentation and Reporting: Maintain thorough documentation for all AI systems developed and provide regular updates to the development team and stakeholders. Requirements: Advanced Degree in Computer Science, AI, Machine Learning, or related field. Proven experience in AI development, preferably in the gaming industry or a similar real-time interactive environment. Strong programming skills in Python, TypeScript, or other relevant languages. Experience with AI frameworks and tools such as TensorFlow, PyTorch, OpenAI GPT, etc. Familiarity with vector databases like Pinecone and experience in handling large-scale data. Strong problem-solving skills and ability to work in a fast-paced, dynamic environment. Excellent communication and teamwork skills. Benefits Salary: You'll earn more than in your previous job. Prime Berlin Office: Our Berlin Mitte office boasts state-of-the-art amenities, including Apple Studio Displays and Herman Miller chairs. Full Commute Reimbursement: We cover 100percet of your commuting costs. Gym Membership: Stay fit and healthy on us. Visa Sponsorship & Assistance: We support your visa process and guide you through all the paperwork. Stock Options: Benefit from our competitive stock option program. Bureaucratic Support: Assistance with German social security, tax forms, and other necessary paperwork.?",
    }
  ]
)

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

message_content = messages[0].content[0].text
print(message_content.value)

Certainly! Here's a rewritten version of your cover letter, tailored for the AI Engineer position at SLAY:

---

Dear SLAY Recruiting Team,

I am thrilled to apply for the AI Engineer position at SLAY. The rapid growth and success of SLAY in redefining social gaming underscore the innovative spirit I am eager to contribute to. My experiences and skills in the fields of augmented reality, machine learning, and game design align seamlessly with SLAY's ambition to revolutionize mobile gaming.

With a B.Sc. in AR/VR/XR Development and Design, I’ve equipped myself with foundational skills in computer graphics, 3D modeling with Blender, and development with Unity【4:0†Nuss-Julia_CV_20241029 Kopie.pdf】. This program allowed me to explore the potential of immersive technologies, driving my passion for creating engaging virtual experiences.

Further enriching my technical portfolio, I recently completed a Data Science and Machine Learning Bootcamp at Ironhack Tech School. This program honed my e

## Exercise 3

Create an assistant that will call a weather API, given the user's answer and return the proper answer.

See the documentation of the weather API here: https://open-meteo.com/en/docs

In [10]:
import requests

def get_weather_forecast(latitude, longitude):
    base_url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m"
    }
    response = requests.get(base_url, params=params)
    return response.json()

# Example usage:
forecast = get_weather_forecast(52.52, 13.41)
print(forecast)

{'latitude': 52.52, 'longitude': 13.419998, 'generationtime_ms': 0.05996227264404297, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 38.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C'}, 'hourly': {'time': ['2024-10-29T00:00', '2024-10-29T01:00', '2024-10-29T02:00', '2024-10-29T03:00', '2024-10-29T04:00', '2024-10-29T05:00', '2024-10-29T06:00', '2024-10-29T07:00', '2024-10-29T08:00', '2024-10-29T09:00', '2024-10-29T10:00', '2024-10-29T11:00', '2024-10-29T12:00', '2024-10-29T13:00', '2024-10-29T14:00', '2024-10-29T15:00', '2024-10-29T16:00', '2024-10-29T17:00', '2024-10-29T18:00', '2024-10-29T19:00', '2024-10-29T20:00', '2024-10-29T21:00', '2024-10-29T22:00', '2024-10-29T23:00', '2024-10-30T00:00', '2024-10-30T01:00', '2024-10-30T02:00', '2024-10-30T03:00', '2024-10-30T04:00', '2024-10-30T05:00', '2024-10-30T06:00', '2024-10-30T07:00', '2024-10-30T08:00', '2024-10-30T09:00', '2024-10-30T10:00', '2024-10-30T11:00', '2024-10-30T12:00'

### If you want to, there is a hint here:

OpenAI Chatbots / Assistants have a way to respond in json format. 

Explore the function calling functionality